In [1]:
from google.colab import drive

MOUNT_PATH = "/content/gdrive"
drive.mount(MOUNT_PATH)

DATA_PATH = MOUNT_PATH + "/My Drive/Datasets/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Very IMPORTANT to install this version as SIFT is patented and not supported in pre-installed versions

!pip install opencv-contrib-python==3.4.2.16

    100% |████████████████████████████████| 30.6MB 1.0MB/s 


In [0]:
import cv2

In [0]:
from sklearn import datasets
import os
import h5py
import six
import cv2
from six.moves import range, cPickle
import tarfile
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets
from scipy import interpolate
import sklearn.svm as svm
from sklearn.naive_bayes import GaussianNB

In [0]:
tar_file = tarfile.open(DATA_PATH + "cifar-10-python.tar.gz", 'r:gz')
train_batches = []
for batch in range(1, 6):
    file = tar_file.extractfile(
        'cifar-10-batches-py/data_batch_%d' % batch)
    try:
        if six.PY3:
            array = cPickle.load(file, encoding='latin1')
        else:
            array = cPickle.load(file)
        train_batches.append(array)
    finally:
        file.close()

train_features = np.concatenate(
    [batch['data'].reshape(batch['data'].shape[0], 3, 32, 32)
        for batch in train_batches])
train_labels = np.concatenate(
    [np.array(batch['labels'], dtype=np.uint8)
        for batch in train_batches])
train_labels = np.expand_dims(train_labels, 1)


In [0]:
file = tar_file.extractfile('cifar-10-batches-py/test_batch')
try:
    if six.PY3:
        test = cPickle.load(file, encoding='latin1')
    else:
        test = cPickle.load(file)
finally:
    file.close()

In [7]:
test_features = test['data'].reshape(test['data'].shape[0], 3, 32, 32)
test_labels = np.array(test['labels'], dtype=np.uint8)
test_labels = np.expand_dims(test_labels, 1)
LABELS = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
          'horse', 'ship', 'truck']
# * 10,000 testing image
print(train_features.shape)
print(test_features.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [0]:
#Images rotated
train_images = np.array([np.rot90(train_features[i].T, k=3) for i in range(0,50000)])           # Train Images
test_images = np.array([np.rot90(test_features[i].T, k=3) for i in range(0,10000)])             # Test Images

In [10]:
train_images.shape

(50000, 32, 32, 3)

In [11]:
hog_extractor = HOG_Extractor()

hog_train_images = []

for image in train_images:
  result = hog_extractor.extract(image)
  hog_train_images.append(result)
  
hog_train_images = np.array(hog_train_images)

hog_train_images.shape

NameError: ignored

In [26]:
hog_test_images = []

for image in test_images:
  result = hog_extractor.extract(image)
  hog_test_images.append(result)
  
hog_test_images = np.array(hog_test_images)

hog_test_images.shape

(10000, 576, 1)

In [30]:
np.array(hog_train_images).reshape(1, -1).shape

(1, 0)

In [14]:
from sklearn import linear_model
from sklearn import metrics
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
lr = linear_model.LogisticRegression()
lr.fit(train_images.reshape(50000, 32 * 32 * 3), train_labels)
 
# Train multinomial logistic regression model
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(train_images.reshape(50000, 576), train_labels)
 
print("Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_labels, lr.predict(train_images.reshape(50000, 32 * 32 * 3))))
print("Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_labels, lr.predict(test_images.reshape(10000, 32 * 32 * 3))))
 
print("Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_labels, mul_lr.predict(train_images.reshape(50000, 32 * 32 * 3))))
print("Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_labels, mul_lr.predict(test_images.reshape(10000, 32 * 32 * 3))))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
lr = linear_model.LogisticRegression()
lr.fit(hog_train_images.reshape(50000, 576), train_labels)
 
# Train multinomial logistic regression model
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
mul_lr.fit(hog_train_images.reshape(50000, 576), train_labels)
 
print("Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_labels, lr.predict(hog_train_images.reshape(50000, 576))))
print("Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_labels, lr.predict(hog_test_images.reshape(10000, 576))))
 
print("Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_labels, mul_lr.predict(hog_train_images.reshape(50000, 576))))
print("Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_labels, mul_lr.predict(hog_test_images.reshape(10000, 576))))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Logistic regression Train Accuracy ::  0.54658
Logistic regression Test Accuracy ::  0.5085
Multinomial Logistic regression Train Accuracy ::  0.55162
Multinomial Logistic regression Test Accuracy ::  0.5101


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

In [0]:
# image_tensor shape : 1 x  3 x 32 x 32
# output image shape :     32 x 32 x  3

def tensor_to_opencv(image_tensor):
  image = (image_tensor.numpy()[0]).astype('uint8') 
  return np.transpose(image.transpose(0, 1, 2))

In [0]:
class HOG_Extractor:
    def __init__(self, window_size = (4,4), block_size = (4,4), block_stride = (1,1), cell_size = (4,4), num_bins = 9, aperture = 1, sigma = 4.0, norm = 0, threshold = 0.5, gamma_correction = 0, num_levels = 64):
        super().__init__()
        self.window_size = window_size
        self.block_size = block_size
        self.block_stride = block_stride
        self.cell_size = cell_size
        self.num_bins = num_bins
        self.aperture = aperture
        self.sigma = sigma
        self.norm = norm
        self.threshold = threshold
        self.gamma_correction = gamma_correction
        self.num_levels = num_levels
        
        self.hog = cv2.HOGDescriptor(window_size,block_size,block_stride,cell_size,num_bins,aperture,sigma,norm,threshold,gamma_correction,num_levels)
  
    def extract(self, image):
      feature = self.hog.compute(image)
      return feature
     

In [0]:
class SIFT_Extractor:
  
  def __init__(self, dictionary_size = 10):
    self.dictionary_size = dictionary_size
    
    self.sift = cv2.xfeatures2d.SIFT_create()
    self.BoW = cv2.BOWKMeansTrainer(dictionary_size)
 

  # Call this with entire data set
  def build_BoW(self, images):
    
  
  # each image shape :     32 x 32 x  3
    
    for image in images:
      key_points, descriptor = self.sift.detectAndCompute(image, None)
      descriptor = np.array(descriptor)
      self.BoW.add(descriptor)
    
    #K-means cluster
    vocabulary = self.BoW.cluster()


    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = self.dictionary_size)
    search_params = dict()  
    flann = cv2.FlannBasedMatcher(index_params,search_params)

    sift2 = cv2.xfeatures2d.SIFT_create()
    bow_dictionary = cv2.BOWImgDescriptorExtractor(sift2, cv2.BFMatcher(cv2.NORM_L2))
    bow_dictionary.setVocabulary(vocabulary)
    
 
  # Feature extraction
  # image_tensor shape : 1 x  3 x 32 x 32  --> PyTorch Tensor
  def extract(self, image_tensor):
    return bow_dictionary.compute(image, self.sift.detect(image))


In [0]:
sift_extractor = SIFT_Extractor()

sift_extractor.build_BoW(hog_train_images)

sift_train_images = np.array([sift_extractor.extract(image) for image in hog_train_images])

sift_test_images = np.array([sift_extractor.extract(image) for image in hog_test_images])